# 🔥 PyTorch: Tensors & Autograd (Silnik Deep Learningu)

Wchodzimy w świat "Prawdziwego AI".
Większość modeli, o których słyszysz (ChatGPT, Stable Diffusion), jest napisana w **PyTorch**.

Zrozumiemy dziś dwa pojęcia:

1.  **Tensor:**
    *   W NumPy mamy `np.array`.
    *   W PyTorch mamy `torch.Tensor`.
    *   Różnica? Tensor można wrzucić na **GPU** (kartę graficzną), co przyspiesza obliczenia 100x.

2.  **Autograd (Automatic Differentiation):**
    *   W Deep Learningu musimy liczyć spadki (gradienty), żeby uczyć sieć.
    *   Liczenie tego ręcznie dla miliona wag jest niemożliwe.
    *   PyTorch "śledzi" każde działanie matematyczne, jakie wykonujesz, i potrafi "odwinąć je do tyłu", żeby policzyć błędy.

In [1]:
# Instalacja (w Colab jest domyślnie, ale na lokalnym PC trzeba zainstalować)
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130

import torch
import numpy as np

print(f"Wersja PyTorch: {torch.__version__}")

Using Python 3.13.2 environment at: venv
Audited 3 packages in 4m 08s


Wersja PyTorch: 2.9.1+cu128


## Krok 1: Tensor vs NumPy

Zobaczysz, że składnia jest prawie identyczna. Jeśli znasz NumPy, znasz PyTorch.
Jedyna różnica to "typ" obiektu.

In [2]:
# 1. Lista Pythonowa
lista = [[1, 2], [3, 4]]

# 2. NumPy Array
np_array = np.array(lista)

# 3. PyTorch Tensor
tensor = torch.tensor(lista)

print(f"NumPy:\n{np_array}")
print(f"Tensor:\n{tensor}")

print("-" * 30)
print("Operacje matematyczne są takie same:")
print(f"Mnożenie (NumPy): \n{np_array * 2}")
print(f"Mnożenie (Torch): \n{tensor * 2}")

NumPy:
[[1 2]
 [3 4]]
Tensor:
tensor([[1, 2],
        [3, 4]])
------------------------------
Operacje matematyczne są takie same:
Mnożenie (NumPy): 
[[2 4]
 [6 8]]
Mnożenie (Torch): 
tensor([[2, 4],
        [6, 8]])


## Krok 2: Magia GPU (CUDA)

To jest powód, dla którego używamy PyTorch.
Zwykła macierz NumPy żyje w RAM-ie i jest liczona przez procesor (CPU).
Tensor możemy wysłać na kartę graficzną (GPU/CUDA).

*Uwaga: Jeśli uruchamiasz to na zwykłym laptopie bez NVIDIA GPU, kod użyje CPU, ale składnia `to(device)` jest uniwersalna.*

In [3]:
# Sprawdzamy, czy mamy dostępną kartę graficzną
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Znaleziono GPU NVIDIA (CUDA)!")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Znaleziono GPU Apple Silicon (M1/M2)!")
else:
    device = torch.device("cpu")
    print("⚠️ Brak GPU. Używamy procesora (CPU).")

# Tworzymy tensor
x = torch.tensor([10.0, 20.0])

# Przenosimy go na urządzenie (to trwa ułamek sekundy, ale fizycznie kopiuje dane)
x = x.to(device)

print(f"Gdzie żyje nasz tensor? {x.device}")
print("Teraz wszystkie obliczenia na 'x' będą super-szybkie.")

✅ Znaleziono GPU NVIDIA (CUDA)!
Gdzie żyje nasz tensor? cuda:0
Teraz wszystkie obliczenia na 'x' będą super-szybkie.


## Krok 3: Autograd (Automatyczne Pochodne)

To jest **najważniejsza część**.
Wyobraź sobie prostą funkcję:
$$ y = x^2 + 5 $$

Chcemy policzyć pochodną (czyli nachylenie/gradient) w punkcie $x=3$.
Z matematyki wiemy, że:
$$ y' = 2x $$
Więc dla $x=3$, wynik powinien wynosić $6$.

Sprawdźmy, czy PyTorch policzy to sam, bez znajomości wzoru na pochodną.

In [4]:
# 1. Definiujemy X
# WAŻNE: requires_grad=True mówi PyTorchowi: "Śledź każdą operację na tej zmiennej!"
x = torch.tensor(3.0, requires_grad=True)

# 2. Definiujemy funkcję Y (Symulacja błędu sieci)
# y = x^2 + 5
y = x**2 + 5

print(f"X: {x}")
print(f"Y (Wynik funkcji): {y}")

# 3. MAGIA: Backward Pass (Wsteczna propagacja)
# Mówimy: "Policz, jak zmiana X wpływa na Y"
y.backward()

# 4. Sprawdzamy wynik
print("-" * 30)
print(f"Matematyka mówi: pochodna z x^2 w punkcie 3 to 2*3 = 6.")
print(f"PyTorch wyliczył: {x.grad}")

if x.grad == 6.0:
    print("✅ Autograd działa idealnie!")

X: 3.0
Y (Wynik funkcji): 14.0
------------------------------
Matematyka mówi: pochodna z x^2 w punkcie 3 to 2*3 = 6.
PyTorch wyliczył: 6.0
✅ Autograd działa idealnie!


## 🧠 Podsumowanie: Po co nam to?

W powyższym przykładzie `y = x^2 + 5`:
*   **x** to Wagi twojej sieci neuronowej (to, co chcemy poprawić).
*   **y** to Błąd sieci (Loss) - np. różnica między tym co sieć przewidziała, a prawdą.

Gdy wywołujesz `loss.backward()`, PyTorch automatycznie liczy:
*"Jak bardzo muszę zmienić wagę X, żeby zmniejszyć błąd Y?"*.

**Tu jest haczyk.**
W GPT-4 masz bilion wag (parametrów).
Ręczne liczenie pochodnych zajęłoby wieczność.
PyTorch robi to automatycznie, budując w pamięci tzw. **Graf Obliczeniowy**. Zapamiętuje, że `y` powstało przez podniesienie `x` do kwadratu, więc wie, jak wrócić po swoich śladach.

W następnym notatniku użyjemy tego, żeby zbudować prawdziwą sieć w stylu **Object Oriented Programming**.